# Estimating Pi

In [1]:
# Import packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import pickle
from pointsgen import *

In [2]:
# Initiate a Points object
if not os.path.exists('..\points.pkl'):
    x = Points(int(1e7), radius = 0.5, xRange = [0, 1], yRange = [0, 1])
    x.df.to_pickle('points.pkl')

#  Read pickle
with open(r'..\points.pkl', 'rb') as file:
    df = pickle.load(file)

# Convert 'inside' to integer type
df['inside'] = df['inside'].astype(int)
df.head()

,x,y,inside
0,0.667262,0.696526,1
1,0.931912,0.608301,1
2,0.105090,0.358075,1
3,0.267763,0.349622,1
4,0.925384,0.705670,1


## Plotting the Randomly Generated Points

In [4]:
# Visualize Monte Carlo
test = df.head(50000)
pointsInside = test['inside'].sum()
title = f'Visualizing {test.shape[0]} Points Inside Square'
fig = go.Figure()
fig.update_layout(width=700, height=700, 
                  title=title, xaxis_title='x', yaxis_title='y', showlegend=False, title_x = 0.5, title_y = 0.9,
                  font=dict(family="Verdana, sans-serif", size=12, color='black'),
                  xaxis=dict(showgrid=False),
                  yaxis=dict(showgrid=False),
                  template = 'simple_white')

# Square
fig.add_shape(type = 'rect', x0=0, y0=0, x1=1, y1=1,
              line = dict(color = 'black'))

# Circle
fig.add_shape(type='circle', x0=0, y0=0,
              x1=1, y1=1,
              line = dict(color = 'black'))

# Scatter Points
fig.add_trace(go.Scatter(x=test['x'], y=test['y'],
    mode='markers',
    marker=dict(
        size=1,
        color= test['inside'], # '#fc6c85'
        colorscale=[[0, '#8fcb9b'], [1, '#5b9279']],
        showscale=False)))

fig.add_annotation(
    text=f"{pointsInside} points\ninside circle",
    xref="paper", yref="paper",
    x=0.5, y=0.5,
    showarrow=False,
    font=dict(size=16, color="black"),
    bgcolor="white",
    opacity = 0.6,
    align="left",
    bordercolor="black",
    borderwidth=1)

fig.add_annotation(
    text=f"{test.shape[0] - pointsInside} points\noutside the circle",
    xref="paper", yref="paper",
    x=1.1, y=0.9,
    showarrow=False,
    font=dict(size=16, color="black"),
    bgcolor="white",
    opacity = 0.6,
    align="right",
    bordercolor="black",
    borderwidth=1)

fig.show()

### How to calculate Pi?
π is calculated by finding the number of points present inside the circle as a proportion of the total points present inside the circle. <br>

\begin{equation}
\pi \approx 4 \cdot \frac{P_{\text{inside}}}{P}
\end{equation}

In [5]:
# Calculating Pi
Pinside = (df['inside'] == 1).sum()
P = df.shape[0]
pi = 4 * Pinside / P
print(pi)

3.1418212


In [29]:
# Accuracy and Precision with # of points
import math

test = test.head(500)
test.index = test.index + 1
x = test.index
test['cumulative'] = test['inside'].cumsum()
test['pi'] = test['cumulative'] * 4 / test.index

fig = go.Figure()
fig.add_trace(go.Scatter(x = x, y = test['pi'], mode = 'lines', name = 'Empirical', line=dict(color='#fc6c85', dash='solid')))

# Update layout with a centered title
fig.update_layout(
    title="Trials vs Accuracy",
    xaxis_title="# of Trials",
    yaxis_title="Accuracy",
    title_x=0.5,
    title_y=0.9,
    template = 'simple_white')

fig.add_trace(go.Scatter(x=[min(x), max(x)], y=[math.pi, math.pi], mode='lines', name='Actual', line=dict(color='grey', dash = 'dot')))
fig.show()